实例演示:
1. 将一个大的excel等拆分成多个excel
2. 将多个小的excel合并为一个大的excel

In [1]:
work_dir='./data/splitTask'
split_dir=f'{work_dir}/splits'
import os
if not os.path.exists(split_dir):
    os.mkdir(split_dir)


0. 读取源excel到pandas

In [2]:
import pandas as pd

In [3]:
df_source=pd.read_excel('./data/movies/users.xlsx')

In [4]:
df_source.head()

,userId,age,gender,occupation,zip-code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [16]:
df_source.loc[0:3]

,userId,age,gender,occupation,zip-code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537


In [7]:
df_source.index

RangeIndex(start=0, stop=943, step=1)

In [8]:
df_source.shape

(943, 5)

In [10]:
# rows计算所有行
rows=df_source.shape[0]
rows

943

一、 将一个大的excel拆分成几个小excel分给若干人的任务

1. 计算拆分后的每个excel的行数

In [11]:
# 这个大excel，会拆分给几个人
task_persons=['wxm','jdq','wyn','yc','xr']

In [12]:
# 每个人的任务数目
splits_size=rows//len(task_persons) if rows%len(task_persons)==0 else rows//len(task_persons)+1

In [13]:
splits_size

189

2. 拆分成多个excel

In [17]:
df_subs=[]

for idx,user_name in enumerate(task_persons):
    # iloc开始的索引
    begin=idx*splits_size
    # iloc结束的索引
    end=begin+splits_size
    print('begin:',begin,'end:',end)
    # 实现df按照iloc拆分
    df_sub=df_source.iloc[begin:end]
    # 将每个子df_sub存入df_subs
    df_subs.append((idx,user_name,df_sub))

begin: 0 end: 189
begin: 189 end: 378
begin: 378 end: 567
begin: 567 end: 756
begin: 756 end: 945


3. 将每个df_sub导出为excel

In [15]:
for idx,user_name,df in df_subs:
    df.to_excel(f'{split_dir}/tasks_{idx}_{user_name}.xlsx',index=False)

二、合并多个小excel到一个大excel
    1. 遍历文件夹，得到要合并的excel文件列表
    2. 分别读取到df中，给每个df添加一列用于标记数据来源
    3. 使用pd.concat进行df批量合并
    4. 将合并后的df输出到excel中

1. 遍历文件夹，得到要合并的excel文件列表

In [18]:
excel_names=[]
for name in os.listdir(split_dir):
    excel_names.append(name)
excel_names

['tasks_0_wxm.xlsx',
 'tasks_1_jdq.xlsx',
 'tasks_2_wyn.xlsx',
 'tasks_3_yc.xlsx',
 'tasks_4_xr.xlsx']

2. 分别读取到df中，给每个df添加一列用于标记数据来源

In [33]:
df_list=[]
# 读取每个excel到df中
for name in excel_names:
    excel_path=f'{split_dir}/{name}'
    df=pd.read_excel(excel_path)
    # 读取到每个任务的姓名
    username=name.replace('tasks_','').replace('.xlsx','')[2:]
    print(username)
    df['name_task']=username
    df_list.append(df)

wxm
jdq
wyn
yc
xr


3. 使用pd.concat进行df批量合并

In [34]:
df_merge=pd.concat(df_list)

In [36]:
df_merge.head()

,userId,age,gender,occupation,zip-code,name_task
0,1,24,M,technician,85711,wxm
1,2,53,F,other,94043,wxm
2,3,23,M,writer,32067,wxm
3,4,24,M,technician,43537,wxm
4,5,33,F,other,15213,wxm


In [38]:
df_merge['name_task'].value_counts()

wxm    189
jdq    189
wyn    189
yc     189
xr     187
Name: name_task, dtype: int64

4. 将合并后的df输出到excel中

In [39]:
df_merge.to_excel(f'{work_dir}/merge_task.xlsx',index=False)